# QuizGenerator

Benvenuti al Generatore di Quiz dedicato al corso "Aspetti Etici, Legali, Sociali ed Economici dell’Informatica" del dipartimento di informatica dell'Università degli Studi di Milano. Questo strumento è stato progettato per consentirti di creare quiz personalizzati, mettendo in primo piano la tua esperienza di apprendimento. Con questo generatore, puoi:

- **Personalizzare il Tuo Quiz**: Scegli il numero di domande che desideri affrontare e seleziona gli argomenti specifici su cui desideri testare la tua preparazione
- **Visualizzare Risultati Dettagliati**: Alla fine del quiz, avrai accesso a risultati dettagliati. Non solo conoscerai il tuo punteggio complessivo, ma potrai anche esplorare i risultati in profondità attraverso grafici informativi. Questi grafici mostreranno la distribuzione delle domande per argomento e la percentuale di risposte corrette per ciascun argomento.

- **Correzione Automatica**: Non preoccuparti della correzione. Questo generatore gestirà automaticamente la verifica delle tue risposte e ti fornirà una correzione accurata per ciascuna domanda.

**Disclaimer**:<br/>
- Le domande contenute in questo generatore sono state estratte da Ariel e proposte nel corso degli anni da altri studenti, con le relative soluzioni<br />
- Questo generatore è concepito come uno strumento complementare, finalizzato a fornire supporto per l'esercizio con domande analoghe a quelle che verranno presentate in sede d'esame
- Non sostituisce lo studio dei materiali del corso disponibili sul sito web ufficiale.

**Versione**: 1.2

In [1]:
from dataclasses import dataclass
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import time
from IPython.display import display, Markdown, Latex, HTML

In [2]:
def selection(message, options, header=None):
    # Verifica se la lista delle opzioni è vuota e solleva un'eccezione se lo è
    if not options:
        raise ValueError("An empty options list is not valid!")

    # Stampa un'intestazione se è stata fornita
    print()
    if header:
        print(header)
        print("-------------------------------")

    # Stampa il messaggio principale
    print(f"{message}\n")

    # Stampa le opzioni numerate
    for i, opt in enumerate(options):
        print(f"\t{i}: {opt}")

    # Inizializza la scelta a un valore non valido
    choice = -1
    while choice < 0 or choice >= len(options):
        try:
            # Ottieni la risposta dall'utente e convertila in un intero
            choice = int(input("Risposta: "))
        except ValueError:
            # Gestisci l'eccezione se l'input non è un numero intero
            choice = -1

    return choice

def time_format(tempo):
    """
    Formatta un valore di tempo in minuti e secondi.

    Argomenti:
    tempo (int): Il valore di tempo in secondi da formattare.

    Restituisce:
    str: Una stringa formattata nel formato "minuti:secondi minuto/minuti".

    Esempi:
    >>> time_format(90)
    '1:30 minuto'
    >>> time_format(120)
    '2:00 minuti'
    >>> time_format(45)
    '0:45 minuti'
    """
    return f'{tempo // 60}:{tempo % 60:02d} minut{"i" if tempo // 60 != 1 else "o"}'

In [3]:
from dataclasses import dataclass

@dataclass
class Question:
    # Attributi della classe Question
    qid: int
    text: str
    options: list
    correct_index: int
    percorso_formativo: str 
    tempo_impiegato: int
    
    def printq(self, choice=-1):
        """
        Stampa la rappresentazione testuale della domanda.
        :param choice: La scelta dell'utente (default: -1).
        """
        print(self.reprq(choice))

    def reprq(self, choice=-1) -> str:
        """
        Restituisce una rappresentazione testuale della domanda.
        :param choice: La scelta dell'utente (default: -1).
        :return: Una stringa con la rappresentazione testuale.
        """
        optresults = ["", "", "", ""]
        if choice >= 0:
            if choice == self.correct_index:
                optresults[choice] = "[CORRECT]"
            else:
                optresults[choice] = "[WRONG]"
                optresults[self.correct_index] = "[CORRECT]"

        return f"""
        <{self.qid}> Question: {self.text}
        ------------------------------------------
        \t0 - {self.options[0]} {optresults[0]}
        \t1 - {self.options[1]} {optresults[1]}
        \t2 - {self.options[2]} {optresults[2]}
        \t3 - {self.options[3]} {optresults[3]}\n\n
        """

    def html_repr(self, choice=-1):
        """
        Restituisce una rappresentazione HTML della domanda.
        :param choice: La scelta dell'utente (default: -1).
        :return: Una stringa con la rappresentazione HTML.
        """
        optresults = ["", "", "", ""]
        if choice >= 0:
            if choice == self.correct_index:
                optresults[choice] = "[CORRECT]"
            else:
                optresults[choice] = "[WRONG]"
                optresults[self.correct_index] = "[CORRECT]"

        return f"""
        <table>
            <thead>
                <tr>
                    <th>{self.qid}</th>
                    <th>{self.text}</th>
                </tr>
            </thead>
            <tbody>
            <tr>
                <td>0</td>
                <td>{self.options[0]}</td>
                <td>{optresults[0]}</td>
            </tr>
            <tr>
                <td>1</td>
                <td>{self.options[1]}</td>
                <td>{optresults[1]}</td>
            </tr>
            <tr>
                <td>2</td>
                <td>{self.options[2]}</td>
                <td>{optresults[2]}</td>
            </tr>
            <tr>
                <td>3</td>
                <td>{self.options[3]}</td>
                <td>{optresults[3]}</td>
            </tr>
            </tbody>
        </table>
        <caption>Tempo impiegato: {time_format(self.tempo_impiegato)}</caption><br />
        <caption>Argomento: {self.percorso_formativo}</caption> 
        <br /><br />
        """


In [4]:
questions = pd.read_csv('./domandeAspetti.tsv', sep='\t', header=0)

## Personalizzazione del quiz
#### Legenda degli argomenti:
  - ECO - Economici
  - LEG - Legali
  - ETH - Etici
  - SOC - SocialiPersonalizzazione del quiz

In [ ]:
# Definizione dei percorsi formativi validi
valid_paths = {"ECO", "SOC", "ETH", "LEG"}

# Imposta iniziale: domande su tutti gli argomenti
selected_paths = ["ECO", "SOC", "ETH", "LEG"]

# Chiedi all'utente se desidera specificare percorsi formativi personalizzati
tmp = input("Vuoi domande su tutti gli argomenti? [si]/no: ")

# Verifica se l'utente ha risposto "no" per personalizzare i percorsi formativi
if tmp.upper() == "NO":
    # Richiedi all'utente di inserire i percorsi formativi desiderati separati da virgola
    selected_paths_input = input("Inserisci i percorsi formativi desiderati separati da virgola (es. ECO, SOC): ")
    
    # Suddividi l'input dell'utente in una lista di percorsi formativi
    selected_paths = list(set(path.strip().upper() for path in selected_paths_input.split(',')))
    
    # Verifica se gli input dell'utente sono validi, altrimenti torna ai percorsi predefiniti
    if len(selected_paths) < 1 or len(selected_paths) > 4 or not all(path in valid_paths for path in selected_paths):
        selected_paths = ["ECO", "SOC", "ETH", "LEG"]

# Calcola la lunghezza delle domande possibili basate sui percorsi formativi selezionati
possible_questions_len = len(questions[questions['PercorsoFormativo'].isin(selected_paths)].index)

# Chiedi all'utente quante domande vuole nel quiz
tmp = input(f"Inserisci quante domande vuoi nel quiz 1-{possible_questions_len} [16]: ")

# Imposta il numero di domande predefinito a 16, a meno che l'utente non fornisca un input valido
QUESTIONS_NUMBER = 16
if tmp.isdigit() and int(tmp) <= possible_questions_len and int(tmp) >= 1:
    QUESTIONS_NUMBER = int(tmp)


In [ ]:
# Estrai indici casuali dal DataFrame "questions" per selezionare domande solo dai percorsi formativi desiderati
index_generated_questions = np.random.choice(
    questions[questions['PercorsoFormativo'].isin(selected_paths)].index,
    QUESTIONS_NUMBER,
    replace=False
)
# Genera un test casuale per ogni domanda con permutazioni delle risposte
generated_test = [
    np.random.permutation(["Risposta0","Risposta1","Risposta2","Risposta3"]) 
    for i in range(QUESTIONS_NUMBER)
]

# Estrai la colonna delle domande dal DataFrame "questions"
dom = questions['Domanda']

# Inizializza una lista vuota per memorizzare i dati delle domande generate
questionsData = []

# Mappatura tra indici risposte corrette e stringa
index_correct_answer = {'Risposta0': 0, 'Risposta1':1, 'Risposta2': 2, 'Risposta3':3}

# Ciclo attraverso il numero di domande da generare
for i in range(QUESTIONS_NUMBER):
    
    # Estrai un indice casuale per una domanda
    randomQ = index_generated_questions[i]
    
    # Estrai la risposta corretta dalla colonna "RispostaCorretta" del DataFrame "questions"
    risposta_corretta = questions.loc[randomQ, 'RispostaCorretta']

    # Estrai il valore "PercorsoFormativo" dalla colonna "PercorsoFormativo" del DataFrame "questions"
    percorso_formativo = questions.loc[randomQ, 'PercorsoFormativo']

    # Estrai e mescola le risposte possibili
    risposte_possibili = np.random.permutation([
        questions.loc[randomQ, 'Risposta0'],
        questions.loc[randomQ, 'Risposta1'],
        questions.loc[randomQ, 'Risposta2'],
        questions.loc[randomQ, 'Risposta3']
    ])
    
    # Crea un oggetto "Question" contenente i dati della domanda
    questionsData.append(
        Question(
            i,  # ID univoco della domanda
            dom[randomQ],  # Testo della domanda estratto dal DataFrame
            risposte_possibili.tolist(),
            np.where(risposte_possibili[:] == questions.loc[randomQ, questions.loc[randomQ, 'RispostaCorretta']])[0][0],  # Indice della risposta corretta
            percorso_formativo,  # Aggiungi l'attributo "percorsoFormativo" al Question object
            0
        )
    )


In [ ]:
# Mostra l'elenco dei percorsi formativi selezionati come una lista non ordinata
formatted_paths = "\n".join([f"- {path}" for path in selected_paths])
display(Markdown(f"<h1>Quiz - {QUESTIONS_NUMBER} domand{'e' if QUESTIONS_NUMBER > 1 else 'a'}</h1>Argomenti:{formatted_paths}"))


In [ ]:
display(Markdown(f"<h1>Quiz - {QUESTIONS_NUMBER} domand{'e' if QUESTIONS_NUMBER > 1 else 'a'}</h1>Argomenti:<ul>{''.join([f'<li>{path}</li>' for path in selected_paths])}</ul>"))

In [ ]:
answers = []
for question in questionsData:
    start = time.time()
    answer = selection(question.text, question.options, header="Domanda N°{}".format(question.qid))
    question.tempo_impiegato = int(time.time() - start)
    answers.append(answer)

# Risultati
I risultati sono proposti come:
- Percentuale risposte esatte sul totale
- Tempo totale impiegato per il quiz
- Distribuzione domande per argomento
- Percenutale risposte esatte per argomento
- Correzione delle domande
- Tempo impiegato per ogni domanda

In [ ]:
# Inizializza i conteggi per le risposte corrette per ciascun percorso formativo
correct_count_by_percorso_formativo = {"ECO": 0, "LEG": 0, "ETH": 0, "SOC": 0, "MISC": 0}

# Inizializza i conteggi totali per ciascun percorso formativo
count_by_percorso_formativo = {"ECO": 0, "LEG": 0, "ETH": 0, "SOC": 0, "MISC": 0}

# Conteggio totale delle risposte corrette
total_correct_count = 0

# Contatore tempo totale impiegato
tempo_totale_impiegato = 0

# Contatore tempo totale impiegato risposte errate e corrette
tempo_totale_impiegato_corrette = 0
tempo_totale_impiegato_errate = 0

# Ciclo attraverso le domande generate
for i in range(QUESTIONS_NUMBER):
    
    # Verifica se la risposta data è corretta
    is_correct = questionsData[i].correct_index == answers[i]

    # Aggiorna il conteggio totale delle risposte corrette
    total_correct_count += is_correct

    # Aggiorna il conteggio delle risposte corrette per il percorso formativo corrispondente
    correct_count_by_percorso_formativo[questionsData[i].percorso_formativo] += is_correct

    # Aggiorna il conteggio totale delle domande per il percorso formativo corrispondente
    count_by_percorso_formativo[questionsData[i].percorso_formativo] += 1

    # Incrementa il contatore del tempo impiegato
    tempo_totale_impiegato += questionsData[i].tempo_impiegato
    if is_correct:
        tempo_totale_impiegato_corrette += questionsData[i].tempo_impiegato
    else:
        tempo_totale_impiegato_errate += questionsData[i].tempo_impiegato


# Calcola la percentuale di risposte corrette
correct_perc = (total_correct_count / len(answers)) * 100

# Calcola media tempo risposte totali,corrette ed errate
tempo_medio_risposta = tempo_totale_impiegato // QUESTIONS_NUMBER
if total_correct_count > 0:
    tempo_medio_corrette = tempo_totale_impiegato_corrette // total_correct_count
else:
    tempo_medio_corrette = 0

if total_correct_count != QUESTIONS_NUMBER:
    tempo_medio_errate = tempo_totale_impiegato_errate // (QUESTIONS_NUMBER - total_correct_count)
else:
    tempo_medio_errate = 0

# Mostra la percentuale di risposte corrette
display(Markdown(f'''
<h3>{"Promosso!" if correct_perc >= (9/16)*100 else "Bocciato!"}</h3>
<ul>
    <li>Risposte corrette: {round(correct_perc, 2)}%</li>
    <li>Tempo totale impiegato: {time_format(tempo_totale_impiegato)}</li>
    <li>Tempo medio di risposta: {time_format(tempo_medio_risposta)}</li>
    <li>Tempo medio risposta corretta: {time_format(tempo_medio_corrette)}</li>
    <li>Tempo medio risposta errata: {time_format(tempo_medio_errate)}</li>
</ul>
'''))


## Grafici
#### Legenda degli argomenti:
  - ECO - Economici
  - LEG - Legali
  - ETH - Etici
  - SOC - Sociali

In [ ]:
# Grafico a torta per la distribuzione delle domande
categorie = [percorso_formativo for percorso_formativo, count in count_by_percorso_formativo.items() if count > 0]
valori = [count for count in count_by_percorso_formativo.values() if count > 0]

plt.pie(valori, labels=categorie, autopct='%1.1f%%', startangle=140)
plt.axis('equal')  # Imposta il grafico a torta in modo circolare
plt.title('Distribuzione delle domande')

# Grafico a barre orizzontali per le percentuali di risposte corrette per argomento
percentuali = [(correct_count_by_percorso_formativo[key] / count_by_percorso_formativo[key]) * 100 for key in categorie]

plt.figure(figsize=(8, 4))  # Imposta la dimensione del grafico a barre
plt.barh(categorie, percentuali, color='green')
plt.xlim(0, 100)  # Imposta i limiti dell'asse x
plt.xlabel('Percentuali risposte corrette per argomento')

# Aggiungi le percentuali come etichette sulle barre
for i, percentuale in enumerate(percentuali):
    plt.text(percentuale, i, f'{int(percentuale)}%', va='center')

# Mostra il grafico a barre orizzontali
plt.show()

## Correzione

In [ ]:
for i in range(QUESTIONS_NUMBER):
    # Mostra l'output HTML della risposta
    display(
        HTML(
            str(questionsData[i].html_repr(answers[i]))
        )
    )